In [1]:
##  Some magic to auto-reload modules
%load_ext autoreload
%autoreload 2

In [6]:
import os
import numpy as np

from scipy.misc import imread, imresize
import pickle 
import pandas as pd

ImportError: No module named pandas

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layersdd.convolutional import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.activations import relu
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.optimizers import SGD, Adagrad
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [4]:
from data_aug import aug_algo

0


In [5]:
train_dir = '/Users/wsm/Downloads/imgs/train/'
test_dir = '/Users/wsm/Downloads/imgs/test/'
titles = ['safe driving', 'texting - right', 'talking on the phone - right', 'texting - left',
          'talking on the phone - left', 'operating the radio', 'drinking', 'reaching behind', 
          'hair and makeup', 'talking to passenger']
driver_img_list = pd.read_csv('driver_imgs_list.csv')
subjectLE = LabelEncoder()
subjectID = subjectLE.fit_transform(driver_img_list.subject.tolist())

classLE = LabelEncoder()
classID = classLE.fit_transform(driver_img_list.classname.tolist())
driver_img_list.loc[:,'subjectID'] = pd.Series(subjectID, index=driver_img_list.index)
driver_img_list.loc[:,'classID'] = pd.Series(classID, index=driver_img_list.index)

NameError: name 'pd' is not defined

In [5]:
nb_classes = 10
img_rows, img_cols = 60, 80
img_channels = 3

In [6]:
def load_train_data(img_list, augment = False):
    train_x = []
    train_y = []
    aug_method = []
    train_subject = []
    algo = aug_algo()
    for i in range(img_list.shape[0]):
        img_name = format('%s/%s/%s' %(train_dir, img_list.iloc[i].classname, img_list.iloc[i].img))
        img = imresize(imread(img_name), (img_rows, img_cols))
        img_y = img_list.classID.iloc[i]
        img_subject = img_list.subjectID.iloc[i]
        train_x += [np.rollaxis(img, 2)]
        train_y += [img_y]
        train_subject += [img_subject]
        aug_method += [0]
        if (augment):
            ## Shift left
                img_shift_left = algo.horizShiftLeft(img, 0.05, 0.15)
                train_x += [np.rollaxis(img_shift_left, 2)]
                train_y += [img_y]
                train_subject += [img_subject]
                aug_method += [1]

            ## Shift Right
                img_shift_right = algo.horizShiftRight(img, 0.05, 0.15)
                train_x += [np.rollaxis(img_shift_right, 2)]
                train_y += [img_y]
                train_subject += [img_subject]
                aug_method += [2]
            
            ## Vertical shift up
                img_shift_up = algo.vertiShiftUp(img, 0.05, 0.15)
                train_x += [np.rollaxis(img_shift_up, 2)]
                train_y += [img_y]
                train_subject += [img_subject]
                aug_method += [3]
                
            ## Vertical shift down
                img_shift_down = algo.vertiShiftDown(img, 0.05, 0.15)
                train_x += [np.rollaxis(img_shift_down, 2)]
                train_y += [img_y]
                train_subject += [img_subject]
                aug_method += [4]
            
            ## Rotate CCW
                img_rotate_cw = algo.rotatedCW(img, 0.15, 0.25, 1.2)
                train_x += [np.rollaxis(img_rotate_cw, 2)]
                train_y += [img_y]
                train_subject += [img_subject]
                aug_method += [5]
            
            ## Rotate CW
                img_rotate_ccw = algo.rotatedCCW(img, 0.15, 0.25, 1.2)
                train_x += [np.rollaxis(img_rotate_ccw, 2)]
                train_y += [img_y]
                train_subject += [img_subject]
                aug_method += [6]
            
            ## Crop and zoom
                img_zoom = algo.cropSkretch(img, 0.05, 0.15)
                train_x += [np.rollaxis(img_zoom, 2)]
                train_y += [img_y]
                train_subject += [img_subject]
                aug_method += [7]
            
        if ((i+1)%3000==0):
            print("Loaded %d" %(i+1))
    
    return (np.array(train_x), np.array(train_y), np.array(train_subject), np.array(aug_method))

In [7]:
def show_img(img):
    plt.imshow(np.rollaxis(np.rollaxis(img,2), 2))

In [ ]:
(train_x, train_y, train_subject, aug_method) = load_train_data(driver_img_list, True)

Loaded 3000
Loaded 6000


In [ ]:
f = open('train6080_aug.save', 'wb')
pickle.dump(a, f, protocol=pickle.HIGHEST_PROTOCOL)
pickle.dump(b, f, protocol=pickle.HIGHEST_PROTOCOL)
pickle.dump(train_y, f, protocol=pickle.HIGHEST_PROTOCOL)
pickle.dump(train_subject, f, protocol=pickle.HIGHEST_PROTOCOL)
pickle.dump(aug_method, f, protocol=pickle.HIGHEST_PROTOCOL)
f.close()

In [ ]:
with open('train6080_aug.save', 'rb') as f:
    a = pickle.load(f)
    b = pickle.load(f)
    train_x = np.concatenate((a,b))
    train_y = pickle.load(f)
    train_y = np_utils.to_categorical(train_y)
    train_subject = pickle.load(f)
    aug_method = pickle.load(f)

    


# Cross validation

In [ ]:
if not load_mask:
	print ("Selecting 3 drivers for validation")
	valid_drivers = np.random.choice(np.unique(train_subject), 3)
	valid_mask = np.in1d(train_subject, valid_drivers)
	with open(mask_path, 'wb') as fmask:
		pickle.dump(valid_mask, fmask, protocol=pickle.HIGHEST_PROTOCOL)
		fmask.close()
else:
	with open(mask_path, 'rb') as fmask:
		valid_mask = pickle.load(fmask)
		fmask.close()

In [ ]:
train_set = train_x[~valid_mask]
valid_set = train_x[np.logical_and(valid_mask, aug_method==0)]

Y_train = train_y[np.flatnonzero(~valid_mask),:]
Y_valid = train_y[np.logical_and(valid_mask, aug_method==0),:]

In [ ]:
Y_valid.shape

In [ ]:
X_train = train_set.astype('float32')
X_train /= 255

X_valid = valid_set.astype('float32')
X_valid /= 255




In [ ]:
X_mean = np.mean(X_train, axis=(0,2,3))
X_train[:,0,:, :] -= X_mean[0]
X_train[:,1,:, :] -= X_mean[1]
X_train[:,2,:, :] -= X_mean[2]


X_valid[:,0,:, :] -= X_mean[0]
X_valid[:,1,:, :] -= X_mean[1]
X_valid[:,2,:, :] -= X_mean[2]

In [ ]:
model = Sequential()

model.add(Convolution2D(64, 3, 3, border_mode='same',
                        input_shape=(img_channels, img_rows, img_cols)))
model.add(Activation('relu'))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Convolution2D(128, 3, 3, border_mode='same'))
model.add(Activation('relu'))

model.add(Convolution2D(128, 3, 3, border_mode='same'))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Convolution2D(256, 3, 3, border_mode='same'))
model.add(Activation('relu'))

model.add(Convolution2D(256, 3, 3, border_mode='same'))
model.add(Activation('relu'))

model.add(Convolution2D(256, 3, 3, border_mode='same'))
model.add(Activation('relu'))


model.add(Convolution2D(256, 3, 3, border_mode='same'))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())

model.add(Dense(1024))
model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(1024))
model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

sgd = SGD(lr=0.03, decay=5e-5, momentum=0.9, nestrov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

In [7]:
model.fit(X_train, Y_train, batch_size=32, nb_epoch=100, shuffle=True,
          validation_data=(X_valid, Y_valid), verbose=1)

NameError: name 'model' is not defined

In [8]:
a = [1,2,3]


In [9]:
a += [4]

In [10]:
a

[1, 2, 3, 4]

In [11]:
a = np.array([1,2,3])


In [12]:
a

array([1, 2, 3])

In [15]:
b = []
b.append(a)

In [19]:
c = [a]
for cc in c:
    print cc

[1 2 3]
